## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/cities.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Montepuez,MZ,-13.1256,38.9997,76.39,51,28,10.42,scattered clouds
1,1,Awjilah,LY,29.1081,21.2869,77.65,29,3,9.01,clear sky
2,2,Busselton,AU,-33.6500,115.3333,53.10,82,30,6.26,scattered clouds
3,3,Georgetown,MY,5.4112,100.3354,80.53,89,40,7.00,scattered clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,61.20,70,27,29.28,scattered clouds
5,5,Florida,US,28.7505,-82.5001,88.92,62,1,5.75,clear sky
6,6,Hermanus,ZA,-34.4187,19.2345,57.90,70,62,8.14,broken clouds
7,7,Henties Bay,NaN,-22.1160,14.2845,62.85,83,0,15.39,clear sky
8,8,Bredasdorp,ZA,-34.5322,20.0403,58.32,72,68,10.36,broken clouds
9,9,Port Blair,IN,11.6667,92.7500,80.42,78,90,1.83,overcast clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp= float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))      

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]>= min_temp)&(city_data_df["Max Temp"]<=max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Montepuez,MZ,-13.1256,38.9997,76.39,51,28,10.42,scattered clouds
1,1,Awjilah,LY,29.1081,21.2869,77.65,29,3,9.01,clear sky
3,3,Georgetown,MY,5.4112,100.3354,80.53,89,40,7.00,scattered clouds
5,5,Florida,US,28.7505,-82.5001,88.92,62,1,5.75,clear sky
9,9,Port Blair,IN,11.6667,92.7500,80.42,78,90,1.83,overcast clouds
13,13,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,73,0,20.71,clear sky
19,19,Mahebourg,MU,-20.4081,57.7000,75.49,78,40,10.36,scattered clouds
20,20,Saint-Philippe,RE,-21.3585,55.7679,75.47,81,81,12.44,broken clouds
21,21,Benjamin Constant,BR,-4.3831,-70.0311,87.93,70,40,8.05,scattered clouds
26,26,Port-Gentil,GA,-0.7193,8.7815,78.28,80,98,9.42,overcast clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df= preferred_cities_df.dropna()

In [19]:
preferred_cities_df.count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Montepuez,MZ,76.39,scattered clouds,-13.1256,38.9997,
1,Awjilah,LY,77.65,clear sky,29.1081,21.2869,
3,Georgetown,MY,80.53,scattered clouds,5.4112,100.3354,
5,Florida,US,88.92,clear sky,28.7505,-82.5001,
9,Port Blair,IN,80.42,overcast clouds,11.6667,92.7500,
13,Arraial Do Cabo,BR,78.76,clear sky,-22.9661,-42.0278,
19,Mahebourg,MU,75.49,scattered clouds,-20.4081,57.7000,
20,Saint-Philippe,RE,75.47,broken clouds,-21.3585,55.7679,
21,Benjamin Constant,BR,87.93,scattered clouds,-4.3831,-70.0311,
26,Port-Gentil,GA,78.28,overcast clouds,-0.7193,8.7815,


In [42]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Montepuez,MZ,76.39,scattered clouds,-13.1256,38.9997,PembaAparthotel
1,Awjilah,LY,77.65,clear sky,29.1081,21.2869,
3,Georgetown,MY,80.53,scattered clouds,5.4112,100.3354,Cititel Penang
5,Florida,US,88.92,clear sky,28.7505,-82.5001,Sportsmens Cove
9,Port Blair,IN,80.42,overcast clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


In [44]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df=hotel_df.dropna()

In [45]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [46]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))